In [52]:
from func.shopee_crawler import search_get_items,search_get_items_with_selected_page, get_items_from_industry, get_in4_item, get_full_in4_item, get_item_price
import os
import pandas as pd

In [53]:
brand_source = os.path.abspath('data\list_requested_brands.xlsx')
# read sheet
# pd.ExcelFile(brand_source).sheet_names
brand_list= [i.replace(' ','%20').lower() for i in list(pd.read_excel(brand_source,'list_brand')['brand'])]

In [54]:
# Lấy danh sách item từ mục tìm kiếm
path = os.path.abspath(r'data\brand_id.xlsx')
data = pd.DataFrame()
for i in brand_list:
    page_number=0
    while True:
        try:
            read_data = pd.read_excel(path)
        except:
            read_data = pd.DataFrame()
        url = f'https://shopee.vn/search?category=11036279&filters=5&keyword={i}&sortBy=sales'
        raw_data = search_get_items_with_selected_page(url=url, page_number=page_number)
        raw_data['f_brand'] = i
        data = pd.concat([data, raw_data], axis=0)
        pd.concat([read_data,raw_data],axis=0).reset_index().drop('index',axis=1).to_excel(path, index=False)
        if data[data['f_brand']==i]['name'].nunique()>100 or page_number>3:
            break
        else:
            page_number +=1
data = data.reset_index().drop('index', axis=1)

In [55]:
#Lấy thông tin chi tiết brand và giá
detail_data = pd.DataFrame()
list_item = data[['shopid','itemid']]
for i in list(range(len(list_item))):
    shopid = int(list_item.iloc[i]['shopid'])
    itemid = int(list_item.iloc[i]['itemid'])
    try:
        raw = get_in4_item(shopid,itemid)
        detail_data = pd.concat([detail_data,raw],axis=0)
    except:
        pass
detail_data = detail_data.reset_index().drop('index', axis=1)
detail_data = detail_data.drop(['shopid','itemid'],axis=1)
# Merge
final = detail_data.merge(data[['shopid','itemid','f_brand']], how='left', left_on = ['shop','item'], right_on = ['shopid','itemid'])
final['price_before_discount'] = final['price_before_discount']/1e5
final['display_price'] = final['display_price']/1e5

In [57]:
final.to_excel(os.path.abspath(r'data\final.xlsx'))